In [1]:
import sys
sys.path.insert(0, 'w:/antpod/GIS/')
import ee
import apgis
import apgis.geemask as mask
import apgis.geebase as base
import geemap

In [2]:
Map = geemap.Map(zoom=1)
Map.add_basemap("SATELLITE")

In [3]:
# Setup

s2 = ee.ImageCollection('COPERNICUS/S2_SR') 
sdate = '2020-01-06'
edate = '2020-01-31'
dList = [sdate, edate]

aoi = ee.Geometry.Polygon([
              [80.056515,12.796234],
              [80.056515,12.828205],
              [80.085268,12.828205],
              [80.085268,12.796234]])

    
s2image = s2.filterDate(dList[0], dList[1]).filterBounds(aoi).first().clip(aoi)
ndvi = s2image.normalizedDifference(['B8','B4']).rename('NDVI')
vis = {'min': 0.0, 'max': 1.0, 'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
                                               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
                                               '004C00', '023B01', '012E01', '011D01', '011301']}
Map.centerObject(aoi, zoom=14)

In [4]:
# Test for function - generateRangeMask

rangemask1 = mask.generateRangeMask(image=ndvi, low=0.5, high= 0.8)
rangemask2 = mask.generateRangeMask(image=ndvi, low=0.2, high= 0.3)

Map.addLayer(rangemask1, vis, 'rangemask1')
Map.addLayer(rangemask2, vis, 'rangemask2')

In [5]:
# Test for function - generateMultiRangeMask

maskcol1 = mask.generateMultiRangeMask(ndvi, [-1, 0.25, 0.33, 0.4, 0.5, 1])
maskcol2 = mask.generateMultiRangeMask(ndvi, [-1, 0.2, 0.31, 0.45, 0.62, 1])

for i in range(5):
    layer1 = 'maskcol1'+ ' Layer ' + str(i)
    layer2 = 'maskcol2'+ ' Layer ' + str(i)
    image1 = base.extractImage(maskcol1, int(i))
    image2 = base.extractImage(maskcol2, int(i))
    Map.addLayer(image1, vis, layer1)
    Map.addLayer(image2, vis, layer2)


In [6]:
Map

Map(center=[12.812219551915618, 80.07089149999587], controls=(WidgetControl(options=['position'], widget=HBox(…